In [36]:
!pip install torch torchvision
!pip install -U finance-datareader


Requirement already up-to-date: finance-datareader in /usr/local/lib/python3.6/dist-packages (0.9.6)


In [0]:
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
import time
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error
import FinanceDataReader as fdr

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
class PredictDataset(Dataset):
    
    def __init__(self, symbol, x_frames):
        
        self.symbol = symbol
        self.x_frames = x_frames
        
        self.start = datetime.datetime.now() - datetime.timedelta(days=(x_frames*2))
        self.end = datetime.date.today()
        self.data = fdr.DataReader(self.symbol, self.start, self.end)
        self.data = self.data.tail(x_frames)
        
    def __len__(self):
        return len(self.data) - (self.x_frames) + 1
    
    def __getitem__(self, idx):
        idx += self.x_frames
        data = self.data.iloc[idx-self.x_frames:idx]
        data = data[['High', 'Low', 'Open', 'Close', 'Change', 'Volume']]
        data = data.apply(lambda x: (x+1) / (x[self.x_frames-1]+1))
        data = data.values
        X = data[:self.x_frames]
        
        return X

In [0]:
class OriginDataset(Dataset):
    
    def __init__(self, symbol, x_frames):
        
        self.symbol = symbol
        self.x_frames = x_frames
        
        self.start = datetime.datetime.now() - datetime.timedelta(days=(x_frames*2))
        self.end = datetime.date.today()
        self.data = fdr.DataReader(self.symbol, self.start, self.end)
        self.data = self.data.tail(x_frames)
        
    def __len__(self):
        return len(self.data) - (self.x_frames) + 1
    
    def __getitem__(self, idx):
        idx += self.x_frames
        data = self.data.iloc[idx-self.x_frames:idx]
        data = data[['High', 'Low', 'Open', 'Close', 'Change', 'Volume']]
        #data = data.apply(lambda x: (x+1) / (x[self.x_frames-1]+1))
        data = data.values
        X = data[:self.x_frames]
        
        return X

In [0]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [42]:
#주식정보들
Symbols = []
with open('/content/gdrive/My Drive/Symbols.txt', 'r') as f:
    data = f.readline()
    Symbols = data.split(',')
print(len(Symbols))

800


In [43]:
batch_size = 1
input_dim = 6
hidden_dim = 50
num_layers = 2
input_frame = 5
output_frame = input_frame
dropout = 1.0
use_bn = True

model = LSTM(input_dim,hidden_dim,output_frame,num_layers,batch_size,dropout,use_bn)
modelPath = '/content/gdrive/My Drive/LSTM_batch_100_model.pt'
model.load_state_dict(torch.load(modelPath))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

LSTM(
  (lstm): LSTM(6, 50, num_layers=2)
  (regressor): Sequential(
    (0): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=1.0, inplace=False)
    (2): Linear(in_features=50, out_features=25, bias=True)
    (3): ReLU()
    (4): Linear(in_features=25, out_features=5, bias=True)
  )
)

In [44]:
predicts = {}
i = 0
for sb in Symbols:
    i += 1
    print(i)
    if i > 10: break
    pset = PredictDataset(sb,input_frame)
    
    model.eval()
    loader = DataLoader(pset,batch_size,shuffle=False)
    for X in loader:
        X = X.transpose(0, 1).float().to(device)
        model.hidden = [hidden.to(device) for hidden in model.init_hidden()]
        y_pred = model(X)
        y_pred = y_pred.view(-1)
        y_pred = np.exp(y_pred.cpu().detach().numpy())
        predicts[sb] = y_pred[-1]


1
2
3
4
5
6
7
8
9
10
11


In [45]:
origins = {}
i = 0
for sb in Symbols:
    i += 1
    print(i)
    if i > 10: break
    oset = OriginDataset(sb,input_frame)
    
    loader = DataLoader(oset,batch_size,shuffle=False)
    for X in loader:
        X = X.transpose(0, 1).float().to(device)
        Xorigin = X[:,:,3].view(-1)
        Xorigin = Xorigin.cpu().detach().numpy()
        origins[sb] = Xorigin[-1]

1
2
3
4
5
6
7
8
9
10
11


In [0]:
result = {}
for i in origins:
    result[i] = int(origins[i] * predicts[i])

In [47]:
!pip install pymysql
import pymysql

In [48]:
conn = pymysql.connect(host='', user='', password='',
                       db='', charset='utf8')
 
# Connection 으로부터 Cursor 생성
curs = conn.cursor()
 

for sb in result:
    #table : stockpreidct
    #symbol, value, type, day
    # SQL문 실행
    sql = """INSERT INTO stockpredict VALUES(%s,%s,%s,NOW())"""
    val = (sb,result[sb],'L')
    curs.execute(sql,val)
    conn.commit()

sql = "select * from stockpredict"
curs.execute(sql)
rows = curs.fetchall()
print(rows)     # 전체 rows

# Connection 닫기
conn.close()

/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'day' at row 1")
  result = self._query(query)


(('353200', 9503, 'L', datetime.date(2020, 6, 9)), ('344820', 38563, 'L', datetime.date(2020, 6, 9)), ('338100', 5184, 'L', datetime.date(2020, 6, 9)), ('308170', 36710, 'L', datetime.date(2020, 6, 9)), ('322000', 26197, 'L', datetime.date(2020, 6, 9)), ('272210', 9638, 'L', datetime.date(2020, 6, 9)), ('317400', 6195, 'L', datetime.date(2020, 6, 9)), ('330590', 6041, 'L', datetime.date(2020, 6, 9)), ('013890', 73556, 'L', datetime.date(2020, 6, 9)), ('336370', 36640, 'L', datetime.date(2020, 6, 9)))
